<div style="display: flex; align-items: left;">
    <a href="https://sites.google.com/corp/google.com/genai-solutions/home?authuser=0">
        <img src="https://storage.googleapis.com/miscfilespublic/Linkedin%20Banner%20%E2%80%93%202.png" style="margin-right">
    </a>
</div>

In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


# **Open Data QnA: Set up Dataset on CloudSQL for PostgreSQL**

---

This notebook shows how to copy a BigQuery public dataset to CloudSQL for PostgreSQL


This is accomplished through the three following steps: 
> i. Set up PostgreSQL instance and databae on Google Cloud SQL

> ii. Copy the BigQuery table to Cloud Storage Bucket

> iii. Create the table in PostgreSQL databae using csv file in Cloud Storage Bucket



### **Change your Kernel to the created .venv with poetry from README.md**

Below is the Kernel how it should look like before you proceed

![Kernel](../utilities/imgs/Kernel%20Changed.png)

## 🔗 **1. Connect Your Google Cloud Project**
Time to connect your Google Cloud Project to this notebook. 

In [ ]:
#@markdown Please fill in the value below with your GCP project ID and then run the cell.
PROJECT_ID = input("Enter the project id (same as your Setup Project) to copy source data in bigquery for this solution")

# Quick input validations.
assert PROJECT_ID, "⚠️ Please provide your Google Cloud Project ID"

# Configure gcloud.
!gcloud config set project {PROJECT_ID}
print(f'Project has been set to {PROJECT_ID}')



## 🔐 **2. Authenticate to Google Cloud**
Authenticate to Google Cloud as the IAM user logged into this notebook in order to access your Google Cloud Project.

You can do this within Google Colab or using the Application Default Credentials in the Google Cloud CLI.

In [ ]:
"""Colab Auth""" 
# from google.colab import auth
# auth.authenticate_user()


"""Jupiter Notebook Auth"""
import google.auth
import os

credentials, project_id = google.auth.default()

os.environ['GOOGLE_CLOUD_QUOTA_PROJECT']=PROJECT_ID
os.environ['GOOGLE_CLOUD_PROJECT']=PROJECT_ID

In [ ]:
#Enable all the required APIs for the COPY

!gcloud services enable \
  cloudapis.googleapis.com \
  compute.googleapis.com \
  iam.googleapis.com \
  sqladmin.googleapis.com \
  bigquery.googleapis.com --project {PROJECT_ID}

## ☁️ **3. Set up Cloud SQL PostgreSQL Instance** 
A **Postgres** Cloud SQL instance is required for the following stages of this notebook.

To connect and access our Postgres Cloud SQL database instance(s) we will leverage the [Cloud SQL Python Connector](https://github.com/GoogleCloudPlatform/cloud-sql-python-connector).

The Cloud SQL Python Connector is a library that can be used alongside a database driver to allow users to easily connect to a Cloud SQL database without having to manually allowlist IP or manage SSL certificates. 

💽 **Create a Postgres Instance**

Running the below cell will verify the existence of a Cloud SQL instance or create a new one if one does not exist.

> ⏳ - Creating a Cloud SQL instance may take a few minutes.

In [ ]:
#@markdown Please fill in the both the Google Cloud region and name of your Cloud SQL instance. Once filled in, run the cell.

# Below are the recommended defaults; These can be changed to values of your choice
PG_REGION = "us-central1" #@param {type:"string"}
PG_INSTANCE = "pg15-opendataqna"
PG_DATABASE = "opendataqna-db"
PG_USER = "pguser"
PG_PASSWORD = "pg123"

# Quick input validations.
assert PG_REGION, "us-central1"
assert PG_INSTANCE, "pg15-opendataqna"

# check if Cloud SQL instance exists in the provided region and create it if it does not exist
database_version = !gcloud sql instances describe {PG_INSTANCE} --format="value(databaseVersion)"
if database_version[0].startswith("POSTGRES"):
  print("Found existing Postgres Cloud SQL Instance!")
else:
  print("Creating new Cloud SQL instance...")
  !gcloud sql instances create {PG_INSTANCE} --database-version=POSTGRES_15 \
    --region={PG_REGION} --cpu=1 --memory=4GB --root-password={PG_PASSWORD} \
    --database-flags=cloudsql.iam_authentication=On

# Create a database on the instance and a user with password
!gcloud sql databases create  {PG_DATABASE} --instance={PG_INSTANCE}
!gcloud sql users create {PG_USER} \
--instance={PG_INSTANCE} \
--password={PG_PASSWORD}

## ➡️ **4. Migrate a public BigQuery database to PostgreSQL instance**
Let's migrate a public BigQuery dataset over to the newly created PostgreSQL instance. 

### A) Set up a Google Cloud Storage Bucket 
This bucket will be used to store the exported BigQuery public dataset.

In [ ]:
from google.cloud import storage
from urllib.error import HTTPError

# Choose a name for the bucket; You might have to choose a different name if the name below already exists
BUCKET_NAME = str(PROJECT_ID+'-opendataqna') #@param {type:"string"} 

# Creating a bucket
storage_client = storage.Client(project=PROJECT_ID)

try: 
    bucket = storage_client.bucket(BUCKET_NAME)

    if bucket.exists(): 
        print("This bucket already exists.")

    else:
        bucket = storage_client.create_bucket(BUCKET_NAME)
        print(f"Bucket {bucket.name} created")

except:
        print("⚠️ This bucket already exists in another project. Make sure to give your bucket a unique name.")

### B) Export BigQuery Dataset to the Bucket


In [ ]:
#@markdown Please choose a BigQuery Public dataset to export. You can leave the default values. Once filled in, run the cell.

# Below are the recommended defaults; You may choose a different database to export
BQ_PROJECT = "bigquery-public-data"
BQ_DATABASE = "census_bureau_international"
bq_tables = [] # Specify empty list to copy 'all' tables, or a Specific list, eg: ["table1", "table3", "table10"]


# Quick input validations.
assert BQ_PROJECT, "⚠️ Please specify the BigQuery Project"
assert BQ_DATABASE, "⚠️ Please specify the BigQuery Database"

from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID)
dataset_id = f'{BQ_PROJECT}.{BQ_DATABASE}'

if not bq_tables:
    bq_tables_obj = client.list_tables(dataset_id)
    bq_tables = [table_obj.table_id for table_obj in bq_tables_obj]

print(f'List of tables in {dataset_id}: {bq_tables}')
destination_uris = []

for bq_table in bq_tables:
    # Export the bigquery data to Google Bucket
    destination_uri = f"gs://{BUCKET_NAME}/{BQ_DATABASE}/{bq_table}.csv"
    dataset_ref = bigquery.DatasetReference(BQ_PROJECT, BQ_DATABASE)
    table_ref = dataset_ref.table(bq_table)

    destination_uris.append(destination_uri)

    extract_job = client.extract_table(
        table_ref,
        destination_uri,
        # Location must match that of the source table.
        location="US",
    )  # API request
    extract_job.result()  # Waits for job to complete.

    print(f"Exported {BQ_PROJECT}:{BQ_DATABASE}.{bq_table} to {destination_uri}")

### C) Retrieve Data Types and Formats 
To migrate the exported .csv files to PostgreSQL, we need to fetch the Data Types and Format from the exported csv file.
This needs to be done as we're setting up the PostgreSQL table and columns first (and need to provide the columns in the setup).
We will load the .csv content into the table afterwards. 

In [ ]:
import pandas as pd

field_names_list = []
field_types_list = []

for destination_uri in destination_uris:

    df = pd.read_csv(destination_uri)
    field_names = df.columns
    field_names_list.append(field_names)
    print(f'Column Names: {field_names}\n')
    field_types = df.dtypes
    field_types_list.append(field_types)
    print(f'Column Names: {field_types}')


### D) Build the SQL Query for Table Creation 
Every database is different. To acommodate for different table structures depending on which BigQuery dataset is being loaded in, we will build the SQL query for creating the required PostgreSQL table dynamically. 

In [ ]:
def get_sql(pg_schema, bq_table, field_names, field_types): 

    cols = "" 

    for i in range(len(field_names)): 
        cols += str(field_names[i]) +" "+ str(field_types[i])
        if i < (len(field_names)-1): 
            cols += ", "


    sql = f"""CREATE TABLE {pg_schema}.{bq_table}({cols})"""

    return sql

#Please specify the PGSchema or leave it as default (public)
PG_SCHEMA = BQ_DATABASE
create_table_sqls = []

for bq_table, field_names, field_types in zip(bq_tables, field_names_list, field_types_list):

    sql = get_sql(PG_SCHEMA, bq_table, field_names, field_types)
    print(f'\nsql for creating {bq_table} PostgreSQL table: \n{sql} \n')
    create_table_sqls.append(sql)


### E) Create the PostgreSQL Table

In [ ]:
import asyncio 
import asyncpg
from google.cloud.sql.connector import Connector

async def create_pg_schema(PROJECT_ID,
                          PG_REGION,
                          PG_INSTANCE,
                          PG_PASSWORD, 
                          PG_DATABASE,
                          PG_USER,
                          PG_SCHEMA): 
    """Delete if PG Schema Exists and create a fresh copy"""
    loop = asyncio.get_running_loop()
    async with Connector(loop=loop) as connector:
        # Create connection to Cloud SQL database
        conn: asyncpg.Connection = await connector.connect_async(
            f"{PROJECT_ID}:{PG_REGION}:{PG_INSTANCE}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{PG_USER}",
            db=f"{PG_DATABASE}",
            password=f"{PG_PASSWORD}"
        )

        await conn.execute(f"DROP SCHEMA IF EXISTS {PG_SCHEMA} CASCADE")        

        await conn.execute(f"CREATE SCHEMA {PG_SCHEMA}")  

        await conn.close()


async def create_pg_table(PROJECT_ID,
                          PG_REGION,
                          PG_INSTANCE,
                          PG_PASSWORD,
                          bq_tables, 
                          PG_DATABASE, 
                          create_table_sqls,
                          PG_USER): 
    """Create PG Table from BQ Schema"""
    
    
    loop = asyncio.get_running_loop()
    async with Connector(loop=loop) as connector:
        # Create connection to Cloud SQL database
        conn: asyncpg.Connection = await connector.connect_async(
            f"{PROJECT_ID}:{PG_REGION}:{PG_INSTANCE}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{PG_USER}",
            db=f"{PG_DATABASE}",
            password=f"{PG_PASSWORD}"
        )

              
        for bq_table, sql in zip(bq_tables, create_table_sqls):
            # Replace the Data Types to work with PostgreSQL supported ones 
            sql = sql.replace("object,", "TEXT,").replace("int64", "INTEGER").replace("float64", "DOUBLE PRECISION")


            await conn.execute(f"DROP TABLE IF EXISTS {bq_table} CASCADE")
            
            # Create the table.
            await conn.execute(sql)

        await conn.close()

# Delete schema if exists and create a fresh copy
await(create_pg_schema(PROJECT_ID, PG_REGION, PG_INSTANCE, PG_PASSWORD, PG_DATABASE, PG_USER, PG_SCHEMA))
# # Create PG Tables
await(create_pg_table(PROJECT_ID, PG_REGION, PG_INSTANCE, PG_PASSWORD, bq_tables, PG_DATABASE, create_table_sqls,PG_USER))

### F) Import Data to PostgreSQL Table
The below cell will iterate through each export file on our Google Cloud Storage Bucket and load it to the PostgreSQL instance. 
This may take a while, depending on the size of the BigQuery public dataset. You can optionally set the LIMIT parameter to limit how many export files will be loaded in. 

In [ ]:
async def import_to_pg(PROJECT_ID,
                          PG_REGION,
                          PG_INSTANCE,
                          PG_USER,
                          PG_PASSWORD,
                          PG_DATABASE,
                          PG_SCHEMA,
                          bq_tables, 
                          BUCKET_NAME): 
    from google.cloud import storage
    import pandas as pd 
    import asyncio
    import asyncpg
    from google.cloud.sql.connector import Connector

    storage_client = storage.Client(project=PROJECT_ID)

    # bucket = storage_client.get_bucket(BUCKET_NAME)
    # blobs = bucket.list_blobs()

    loop = asyncio.get_running_loop()
    async with Connector(loop=loop) as connector:


        # Create connection to Cloud SQL database
        conn: asyncpg.Connection = await connector.connect_async(
            f"{PROJECT_ID}:{PG_REGION}:{PG_INSTANCE}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{PG_USER}",
            password=f"{PG_PASSWORD}",
            db=f"{PG_DATABASE}",
        )
       
        for bq_table in bq_tables:
            URI = f"gs://{BUCKET_NAME}/{PG_SCHEMA}/{bq_table}.csv"
            print(f'URI:{URI}')
            df = pd.read_csv(URI)
            df = df.dropna()
            df.info()   

            # Copy the dataframe to the table.
            tuples = list(df.itertuples(index=False))

            await conn.copy_records_to_table(
                bq_table, records=tuples, columns=list(df), schema_name=PG_SCHEMA, timeout=3600
            )
        await conn.close()

# # Load Data into PG Table 
await(import_to_pg(PROJECT_ID, PG_REGION, PG_INSTANCE, PG_USER, PG_PASSWORD, PG_DATABASE, PG_SCHEMA,bq_tables, BUCKET_NAME))

### If all the above steps are executed suucessfully, the Bigquery public dataset should be copied to Cloud SQL for PostgreSQL on your GCP project